In [ ]:
pip install tensorflow==2.3.2

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Set image dimensions



In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 1
NUM_CLASSES = 5

# Data paths
train_dir = "Data/train"
val_dir = "Data/validation"

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)




In [ ]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)


In [ ]:
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',  # FIXED
              metrics=['accuracy'])


In [ ]:
model.fit(train_generator,
          steps_per_epoch=100,
          epochs=EPOCHS)

# Save model
model.save("rice.h5")
